Связь многие ко многим (many to many) это тип связи, когда нет родительских и дочерних таблиц. Любой записи таблицы А могут соответствовать несколько записей таблицы Б, и в то же время любой записи таблицы Б могут соответствовать несколько записей таблицы А.

Для создания такого типа связи используют вспомогательную таблицу в которой ключевое поле таблицы А соотносится с ключевым полем таблицы Б.

Создадим главные таблицы, между которыми будет связь многие ко многим.

In [1]:
import sqlite3

# Создаем основные таблицы
with sqlite3.connect(':memory:') as db:
    try:
        cursor = db.cursor()
        cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS tags (id INTEGER PRIMARY KEY AUTOINCREMENT, tag_name TEXT);
        """
        )

        cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS posts (id INTEGER PRIMARY KEY AUTOINCREMENT, post_title TEXT, post_content TEXT);
        """
        )
    except Exception as e:
        print(e)
    finally:
        cursor.close()

Создадим вспомогательную таблицу, которая будет содержать связи между записями в главных таблицах.

Здесь мы создаем перекрестные ссылки и указываем ограничение `CONSTRAIT` на первичный ключ `tags_post_pk`, чтобы его можно было создавать только если существуют записи `tag_id` и `post_id`.

In [2]:
# Создадим вспомогательную таблицу
try:
    cursor = db.cursor()
    cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS tags_posts (tag_id INTEGER, post_id INTEGER,
    FOREIGN KEY (tag_id) REFERENCES posts (id),
    FOREIGN KEY (post_id) REFERENCES tags (id),
    CONSTRAINT tags_post_pk PRIMARY KEY (tag_id, post_id));
    """
    )
except Exception as e:
    print(e)
finally:
    cursor.close()

Заполним главные таблицы такими данными, которые имеют между собой связь многие к многим.

In [3]:
# Заполним основные таблицы данными
tags = [
    ('Flask',),
    ('Django',),
    ('Docker',),
]
posts = [
    ('Docker и Flask', 'Контейнеризация приложений Flask'),
    ('Docker и Django', 'Контейнеризация приложений Django'),
    ('Основы Flask', 'Основы фреймворка Flask'),
    ('Основы Django', 'Основы фреймворка Django'),
    ('Основы Docker', 'Основы Docker'),
]

try:
    cursor = db.cursor()
    cursor.executemany(
    """
    INSERT INTO tags (tag_name) VALUES (?);
    """,
    tags
    )
    cursor.executemany(
    """
    INSERT INTO posts (post_title, post_content) VALUES (?, ?);
    """,
    posts
    )
except Exception as e:
    print(e)
finally:
    cursor.close()

Заполним вспомогательную таблицу.

При этом если мы допустим ошибку, указав несуществующий ключ для любой из главных таблиц, то связь просто не будет создана.

In [4]:
# Заполним вспомогательную таблицу связями
links = [
    (1, 1),
    (1, 3),
    (2, 2),
    (2, 4),
    (3, 1),
    (3, 2),
    (3, 5),
]

try:
    cursor = db.cursor()
    cursor.executemany(
    """
    INSERT INTO tags_posts (tag_id, post_id) VALUES (?, ?);
    """,
    links
    )
except Exception as e:
    print(e)
finally:
    cursor.close()

Самым главным является правильно оформить запрос к базе данных.

Мы выбираем нужные поля из таблиц (строка 1), затем выбираем записи из `tags_posts` которые относятся к одному `post.id` (строка 2) и уже из этих записей выбирает те, у которых `tag_id` относится к отдельному тегу (строка 3).

In [9]:
# Сделаем выборку записей из таблиц
try:
    cursor = db.cursor()
    data = cursor.execute(
    """
    SELECT posts.post_title, posts.post_content, tags.tag_name FROM posts
    LEFT JOIN tags_posts ON tags_posts.post_id = posts.id
    LEFT JOIN tags ON tags_posts.tag_id = tags.id;
    """
    ).fetchall()
    for i in data:
        print(i)
except Exception as e:
    print(e)
finally:
    cursor.close()

('Docker и Flask', 'Контейнеризация приложений Flask', 'Flask')
('Docker и Flask', 'Контейнеризация приложений Flask', 'Docker')
('Docker и Django', 'Контейнеризация приложений Django', 'Django')
('Docker и Django', 'Контейнеризация приложений Django', 'Docker')
('Основы Flask', 'Основы фреймворка Flask', 'Flask')
('Основы Django', 'Основы фреймворка Django', 'Django')
('Основы Docker', 'Основы Docker', 'Docker')
